In [1]:
!pip install transformers datasets torch scikit-learn pandas numpy tqdm

In [1]:
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

In [2]:
# ============================================================
# JUDICIAL-SPECIFIC TRAIT TAXONOMY (50 traits)
# ============================================================

JUDICIAL_TRAITS = {
    # ─── COGNITIVE BIASES (10 traits) ───────────────────────
    "confirmation_bias": "Selectively citing precedent that supports predetermined conclusion",
    "anchoring_bias": "Over-relying on initial information or first argument presented",
    "hindsight_bias": "Viewing past events as more predictable than they actually were",
    "status_quo_bias": "Preferring existing legal frameworks over novel interpretations",
    "recency_bias": "Giving disproportionate weight to recent cases over established precedent",
    "availability_heuristic": "Over-weighting vivid or memorable cases in reasoning",
    "framing_effect": "Outcome depends on how facts are presented (victim vs defendant focus)",
    "sunk_cost_fallacy": "Continuing flawed reasoning because of prior investment in position",
    "authority_bias": "Excessive deference to higher courts or expert witnesses",
    "outcome_bias": "Judging decision quality by result rather than process",

    # ─── DEMOGRAPHIC BIASES (8 traits) ──────────────────────
    "racial_bias_criminal": "Harsher language/outcomes for defendants of certain races",
    "racial_bias_civil": "Differential treatment in civil matters based on race",
    "gender_bias_criminal": "Gender-based disparities in sentencing language",
    "gender_bias_civil": "Gender stereotypes in family law, employment, harassment cases",
    "socioeconomic_bias": "Different standards for wealthy vs poor litigants",
    "religious_bias": "Favoritism or prejudice based on religious affiliation",
    "age_bias": "Differential treatment based on age (youth vs elderly)",
    "immigration_status_bias": "Bias against non-citizens or undocumented persons",

    # ─── IDEOLOGICAL/POLITICAL (8 traits) ───────────────────
    "conservative_criminal": "Pro-prosecution, law-and-order stance in criminal cases",
    "liberal_criminal": "Pro-defendant, civil liberties emphasis in criminal cases",
    "conservative_civil": "Pro-business, limited government in civil/regulatory cases",
    "liberal_civil": "Pro-plaintiff, expansive rights interpretation",
    "textualist": "Strict adherence to statutory/constitutional text",
    "purposivist": "Interpreting law based on legislative intent/purpose",
    "judicial_activist": "Willingness to overturn precedent or expand rights",
    "judicial_restraint": "Deference to legislature, narrow rulings",

    # ─── REASONING QUALITY (10 traits) ──────────────────────
    "logical_reasoning": "Clear, coherent logical progression",
    "precedent_mastery": "Thorough, accurate citation and application of precedent",
    "factual_analysis": "Careful attention to factual record",
    "legal_scholarship": "Deep engagement with legal theory and scholarship",
    "policy_consideration": "Explicit weighing of policy implications",
    "oversimplification": "Ignoring complexity or nuance in law/facts",
    "circular_reasoning": "Conclusions that assume what they're trying to prove",
    "strawman_argument": "Mischaracterizing opposing arguments",
    "red_herring": "Introducing irrelevant considerations",
    "selective_precedent": "Cherry-picking cases while ignoring contrary authority",

    # ─── TEMPERAMENT/STYLE (8 traits) ───────────────────────
    "empathetic_tone": "Acknowledging human impact and suffering",
    "clinical_detachment": "Emotionally neutral, purely analytical",
    "moral_outrage": "Expressing strong moral judgment",
    "defensive_posture": "Anticipating and rebutting criticism",
    "collegial": "Respectful treatment of opposing views",
    "dismissive": "Contemptuous treatment of arguments",
    "verbose": "Unnecessarily lengthy or complex writing",
    "concise_clarity": "Clear, efficient communication",

    # ─── PROCEDURAL TENDENCIES (6 traits) ───────────────────
    "pro_plaintiff_procedure": "Resolving procedural ambiguities in plaintiff's favor",
    "pro_defendant_procedure": "Strict procedural requirements favoring defendants",
    "formalist": "Strict adherence to procedural rules",
    "pragmatic": "Flexible procedure to achieve substantive justice",
    "deferential_to_jury": "Upholding jury verdicts absent clear error",
    "skeptical_of_jury": "Frequently overturning jury decisions"
}

In [3]:
TRAIT_NAMES = list(JUDICIAL_TRAITS.keys())
NUM_TRAITS = len(TRAIT_NAMES)
TRAIT_TO_IDX = {t: i for i, t in enumerate(TRAIT_NAMES)}

In [4]:
# ============================================================
# TRAINING DATA SOURCES (Real Legal Text)
# ============================================================

class JudicialDatasetBuilder:
    """Build training dataset from real judicial opinions."""

    def __init__(self):
        self.data = []

    def load_courtlistener_opinions(self, n_samples: int = 10000):
        """
        Load from CourtListener API (free, public judicial opinions)
        https://www.courtlistener.com/api/
        """
        # Example: Federal Circuit opinions from 2020-2024
        # You'll need to implement the actual API calls

        opinions = [
            {
                "text": "Opinion text here...",
                "case_name": "Smith v. Jones",
                "judge": "Judge Roberts",
                "court": "9th Circuit",
                "year": 2023,
                "case_type": "criminal",
                "outcome": "reversed"
            }
            # ... load real opinions
        ]

        return opinions

    def load_scotus_corpus(self):
        """
        Supreme Court opinions are perfect for bias analysis.
        Use the Supreme Court Database or Oyez Project.
        """
        # Known ideological splits are great labels
        # Conservative: Thomas, Alito, Gorsuch, Kavanaugh, Barrett
        # Liberal: Sotomayor, Kagan, Jackson
        # Swing: Roberts

        pass

    def load_labeled_examples_from_research(self):
        """
        Academic papers on judicial bias often include labeled examples.

        Sources:
        - "Computational Analysis of Racial Disparities in Judicial Opinions"
        - "Gender Bias in Federal Sentencing"
        - "Measuring Judicial Ideology"
        """

        # Example labeled data from research papers
        labeled_examples = [
            {
                "text": "The defendant, who has a lengthy criminal history...",
                "labels": {
                    "confirmation_bias": 1,
                    "conservative_criminal": 1,
                    "hindsight_bias": 1
                }
            },
            {
                "text": "While the plaintiff's claims have some merit, the procedural...",
                "labels": {
                    "pro_defendant_procedure": 1,
                    "formalist": 1,
                    "dismissive": 0
                }
            }
        ]

        return labeled_examples

    def generate_synthetic_judicial_examples(self, trait: str, n: int = 100):
        """
        Use Claude to generate realistic judicial opinion excerpts.
        This is MORE effective than generic text because we prompt
        specifically for legal language patterns.
        """

        #import anthropic
        #client = anthropic.Anthropic()  # Add your API key
        from vertexai.generative_models import GenerativeModel, Part
        import vertexai
        from google.colab import auth
        auth.authenticate_user()

        LOCATION = "us-central1"
        PROJECT_ID = "personality-detection-486205" # Replace this!
        # Initialize Vertex AI (Ensure your Project ID and Location are set)
        vertexai.init(project=PROJECT_ID, location=LOCATION)
        model = GenerativeModel("gemini-2.5-pro")
        prompt = f"""You are a legal scholar analyzing judicial bias. Generate {n} realistic excerpts from judicial opinions that exhibit: {trait}

Definition: {JUDICIAL_TRAITS[trait]}

Requirements:
1. Use authentic legal language (whereas, heretofore, plaintiff/defendant, etc.)
2. Cite fictional precedents in proper format (e.g., Smith v. Jones, 123 F.3d 456 (9th Cir. 2020))
3. Include legal reasoning patterns typical of judges
4. Vary between trial and appellate opinions
5. Vary case types (criminal, civil, constitutional)
6. Each excerpt should be 2-4 sentences
7. Make the bias SUBTLE (real judicial bias is rarely overt)

Return ONLY a JSON array of strings:
["excerpt 1", "excerpt 2", ...]"""

        '''response = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=4000,
            messages=[{"role": "user", "content": prompt}]
        )
        '''
        response = model.generate_content(prompt)
        text = response.text.strip()
        if "```" in text:
            text = text.split("```")[1].replace("json", "").strip()

        examples = json.loads(text)
        return examples

    def build_complete_dataset(self):
        """Orchestrate all data sources."""

        print("Building judicial bias training dataset...")

        # 1. Generate synthetic examples for each trait (fast, controlled)
        print("\n[1/4] Generating synthetic judicial examples...")
        for trait in TRAIT_NAMES:
            print(f"  Generating {trait}...")
            examples = self.generate_synthetic_judicial_examples(trait, n=50)

            for text in examples:
                label_vector = [0] * NUM_TRAITS
                label_vector[TRAIT_TO_IDX[trait]] = 1
                self.data.append({
                    "text": text,
                    "labels": label_vector,
                    "source": "synthetic"
                })

        print(f"  ✅ Synthetic: {len(self.data)} examples")

        # 2. Load real opinions from CourtListener
        print("\n[2/4] Loading CourtListener opinions...")
        # opinions = self.load_courtlistener_opinions()
        # Apply heuristic labeling based on case metadata

        # 3. Load SCOTUS opinions with known ideological labels
        print("\n[3/4] Loading Supreme Court opinions...")
        # scotus_data = self.load_scotus_corpus()

        # 4. Load manually labeled examples from research
        print("\n[4/4] Loading research-labeled examples...")
        # research_data = self.load_labeled_examples_from_research()

        print(f"\n✅ Total dataset: {len(self.data)} examples")
        return self.data


In [5]:
# ============================================================
# SPECIALIZED MODEL ARCHITECTURE
# ============================================================

class JudicialBiasClassifier(nn.Module):
    """
    Legal-BERT fine-tuned for judicial bias detection.
    Uses legal domain pretrained model for better understanding.
    """

    def __init__(self, num_traits: int):
        super().__init__()

        # Use Legal-BERT instead of generic BERT
        # It's pretrained on legal documents (case law, contracts, statutes)
        self.backbone = AutoModel.from_pretrained(
            "nlpaueb/legal-bert-base-uncased"
            # Alternative: "pile-of-law/legalbert-large-1.7M-2"
        )

        hidden_size = self.backbone.config.hidden_size  # 768

        # Multi-label classification head
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_traits)

    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Use [CLS] token representation
        pooled = outputs.last_hidden_state[:, 0, :]
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)

        return logits

In [6]:
# ============================================================
# INFERENCE: ANALYZE A JUDGE'S OPINION
# ============================================================

class JudgeProfiler:
    """Analyze judicial opinions to build bias/personality profile."""

    def __init__(self, model_path: str):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load model
        self.model = JudicialBiasClassifier(NUM_TRAITS)
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.to(self.device)
        self.model.eval()

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

    def analyze_opinion(self, opinion_text: str, threshold: float = 0.3) -> Dict:
        """
        Analyze a single judicial opinion.

        Returns:
            {
                "traits": {trait: confidence_score},
                "profile_summary": str,
                "red_flags": List[str]
            }
        """

        # Tokenize (split long opinions into chunks)
        chunks = self._chunk_text(opinion_text, max_length=512)

        all_predictions = []

        for chunk in chunks:
            encoding = self.tokenizer(
                chunk,
                truncation=True,
                max_length=512,
                padding="max_length",
                return_tensors="pt"
            )

            input_ids = encoding["input_ids"].to(self.device)
            attention_mask = encoding["attention_mask"].to(self.device)

            with torch.no_grad():
                logits = self.model(input_ids, attention_mask)
                probs = torch.sigmoid(logits).cpu().numpy()[0]
                all_predictions.append(probs)

        # Average predictions across chunks
        avg_probs = np.mean(all_predictions, axis=0)

        # Extract traits above threshold
        detected_traits = {}
        for idx, score in enumerate(avg_probs):
            if score >= threshold:
                detected_traits[TRAIT_NAMES[idx]] = float(score)

        # Generate profile summary
        profile = self._generate_profile_summary(detected_traits)
        red_flags = self._identify_red_flags(detected_traits)

        return {
            "traits": detected_traits,
            "profile_summary": profile,
            "red_flags": red_flags,
            "raw_scores": {TRAIT_NAMES[i]: float(avg_probs[i]) for i in range(NUM_TRAITS)}
        }

    def analyze_judge_corpus(self, opinions: List[str]) -> Dict:
        """
        Analyze multiple opinions from same judge to build aggregate profile.

        This is MORE reliable than single-opinion analysis.
        """

        all_traits = []

        for opinion in opinions:
            result = self.analyze_opinion(opinion, threshold=0.25)
            all_traits.append(result["raw_scores"])

        # Aggregate across all opinions
        avg_scores = {}
        for trait in TRAIT_NAMES:
            scores = [t[trait] for t in all_traits]
            avg_scores[trait] = np.mean(scores)

        # Identify consistent patterns (high average + low variance)
        consistent_traits = {}
        for trait in TRAIT_NAMES:
            scores = [t[trait] for t in all_traits]
            if np.mean(scores) > 0.4 and np.std(scores) < 0.2:
                consistent_traits[trait] = {
                    "mean": float(np.mean(scores)),
                    "std": float(np.std(scores)),
                    "appearances": sum(1 for s in scores if s > 0.3)
                }

        return {
            "aggregate_profile": avg_scores,
            "consistent_patterns": consistent_traits,
            "total_opinions_analyzed": len(opinions),
            "profile_summary": self._generate_profile_summary(consistent_traits)
        }

    def compare_judges(self, judge_a_opinions: List[str],
                       judge_b_opinions: List[str]) -> Dict:
        """Compare two judges' bias profiles."""

        profile_a = self.analyze_judge_corpus(judge_a_opinions)
        profile_b = self.analyze_judge_corpus(judge_b_opinions)

        # Compute difference vectors
        differences = {}
        for trait in TRAIT_NAMES:
            diff = profile_a["aggregate_profile"][trait] - profile_b["aggregate_profile"][trait]
            if abs(diff) > 0.2:  # Meaningful difference
                differences[trait] = {
                    "difference": float(diff),
                    "judge_a": profile_a["aggregate_profile"][trait],
                    "judge_b": profile_b["aggregate_profile"][trait]
                }

        return {
            "judge_a_profile": profile_a,
            "judge_b_profile": profile_b,
            "significant_differences": differences
        }

    def _chunk_text(self, text: str, max_length: int = 512) -> List[str]:
        """Split long opinions into overlapping chunks."""
        words = text.split()
        chunks = []
        stride = max_length - 50  # 50 word overlap

        for i in range(0, len(words), stride):
            chunk = " ".join(words[i:i+max_length])
            if len(chunk.split()) > 50:  # Skip tiny fragments
                chunks.append(chunk)

        return chunks if chunks else [text]

    def _generate_profile_summary(self, traits: Dict) -> str:
        """Generate human-readable summary."""
        if not traits:
            return "No significant bias patterns detected."

        # Group by category
        bias_traits = [t for t in traits if "bias" in t]
        ideological = [t for t in traits if "conservative" in t or "liberal" in t]
        reasoning = [t for t in traits if t in ["logical_reasoning", "precedent_mastery", "oversimplification"]]

        summary_parts = []

        if bias_traits:
            summary_parts.append(f"⚠️ Demographic biases: {', '.join(bias_traits[:3])}")

        if ideological:
            summary_parts.append(f"🔵/🔴 Ideological: {', '.join(ideological)}")

        if reasoning:
            summary_parts.append(f"📊 Reasoning quality: {', '.join(reasoning)}")

        return " | ".join(summary_parts) if summary_parts else "Standard judicial profile."

    def _identify_red_flags(self, traits: Dict) -> List[str]:
        """Flag concerning bias patterns."""
        red_flags = []

        # Demographic biases are always red flags
        demographic_biases = [
            "racial_bias_criminal", "racial_bias_civil",
            "gender_bias_criminal", "gender_bias_civil",
            "socioeconomic_bias", "religious_bias"
        ]

        for bias in demographic_biases:
            if traits.get(bias, 0) > 0.5:
                red_flags.append(f"Strong {bias} detected (score: {traits[bias]:.2f})")

        # Reasoning quality issues
        if traits.get("circular_reasoning", 0) > 0.6:
            red_flags.append("Frequent circular reasoning patterns")

        if traits.get("selective_precedent", 0) > 0.6:
            red_flags.append("Cherry-picking precedent")

        return red_flags


In [7]:
# ============================================================
# EXPORT TO CSV WITH PROPER HEADERS
# ============================================================

def export_analysis_to_csv(results: Dict, output_path: str):
    """
    Export judge analysis to CSV with proper headers.

    Format:
    Judge,Opinion_ID,confidence_bias,logical_reasoning,...,Profile_Summary
    """

    rows = []

    for opinion_id, analysis in results.items():
        row = {
            "Opinion_ID": opinion_id,
            "Profile_Summary": analysis["profile_summary"]
        }

        # Add all trait scores
        for trait in TRAIT_NAMES:
            row[trait] = analysis["raw_scores"].get(trait, 0.0)

        rows.append(row)

    df = pd.DataFrame(rows)

    # Reorder columns: metadata first, then traits alphabetically
    metadata_cols = ["Opinion_ID", "Profile_Summary"]
    trait_cols = sorted([c for c in df.columns if c not in metadata_cols])
    df = df[metadata_cols + trait_cols]

    df.to_csv(output_path, index=False)
    print(f"✅ Exported to {output_path}")
    print(f"   {len(df)} opinions analyzed")
    print(f"   {len(trait_cols)} trait columns")



 STEP 1: BUILD DATASET
Building judicial bias training dataset...

[1/4] Generating synthetic judicial examples...
  Generating confirmation_bias...


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


  Generating anchoring_bias...
  Generating hindsight_bias...
  Generating status_quo_bias...
  Generating recency_bias...
  Generating availability_heuristic...
  Generating framing_effect...
  Generating sunk_cost_fallacy...
  Generating authority_bias...
  Generating outcome_bias...
  Generating racial_bias_criminal...
  Generating racial_bias_civil...
  Generating gender_bias_criminal...
  Generating gender_bias_civil...
  Generating socioeconomic_bias...
  Generating religious_bias...
  Generating age_bias...
  Generating immigration_status_bias...
  Generating conservative_criminal...
  Generating liberal_criminal...
  Generating conservative_civil...
  Generating liberal_civil...
  Generating textualist...
  Generating purposivist...
  Generating judicial_activist...
  Generating judicial_restraint...
  Generating logical_reasoning...
  Generating precedent_mastery...
  Generating factual_analysis...
  Generating legal_scholarship...
  Generating policy_consideration...
  Genera

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: nlpaueb/legal-bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.decoder.weight             | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.decoder.bias               | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


FileNotFoundError: [Errno 2] No such file or directory: 'judicial_bias_model.pt'

In [8]:
import json
import pandas as pd

# Define the trait taxonomy
JUDICIAL_TRAITS = {
    # COGNITIVE BIASES (10 traits)
    "confirmation_bias": 0, "anchoring_bias": 1, "hindsight_bias": 2,
    "status_quo_bias": 3, "recency_bias": 4, "availability_heuristic": 5,
    "framing_effect": 6, "sunk_cost_fallacy": 7, "authority_bias": 8,
    "outcome_bias": 9,

    # DEMOGRAPHIC BIASES (8 traits)
    "racial_bias_criminal": 10, "racial_bias_civil": 11, "gender_bias_criminal": 12,
    "gender_bias_civil": 13, "socioeconomic_bias": 14, "religious_bias": 15,
    "age_bias": 16, "immigration_status_bias": 17,

    # IDEOLOGICAL/POLITICAL (8 traits)
    "conservative_criminal": 18, "liberal_criminal": 19, "conservative_civil": 20,
    "liberal_civil": 21, "textualist": 22, "purposivist": 23,
    "judicial_activist": 24, "judicial_restraint": 25,

    # REASONING QUALITY (10 traits)
    "logical_reasoning": 26, "precedent_mastery": 27, "factual_analysis": 28,
    "legal_scholarship": 29, "policy_consideration": 30, "oversimplification": 31,
    "circular_reasoning": 32, "strawman_argument": 33, "red_herring": 34,
    "selective_precedent": 35,

    # TEMPERAMENT/STYLE (8 traits)
    "empathetic_tone": 36, "clinical_detachment": 37, "moral_outrage": 38,
    "defensive_posture": 39, "collegial": 40, "dismissive": 41,
    "verbose": 42, "concise_clarity": 43,

    # PROCEDURAL TENDENCIES (6 traits)
    "pro_plaintiff_procedure": 44, "pro_defendant_procedure": 45, "formalist": 46,
    "pragmatic": 47, "deferential_to_jury": 48, "skeptical_of_jury": 49
}

NUM_TRAITS = 50

# Load your training data
print("Loading training data...")
with open('judicial_training_data.json', 'r') as f:
    data = json.load(f)

# Extract texts and labels
texts = [item['text'] for item in data]
labels = [item['labels'] for item in data]

print(f"✓ Loaded {len(texts)} training examples")
print(f"✓ Each example has {len(labels[0])} bias traits")
print(f"✓ Expected {NUM_TRAITS} traits")

# Verify label dimensions
assert len(labels[0]) == NUM_TRAITS, f"Label mismatch! Expected {NUM_TRAITS}, got {len(labels[0])}"

print(f"\n📄 First example text: {texts[0][:150]}...")
print(f"📊 First example labels sum: {sum(labels[0])} traits detected")

Loading training data...
✓ Loaded 2500 training examples
✓ Each example has 50 bias traits
✓ Expected 50 traits

📄 First example text: The exclusionary rule’s primary purpose is deterrence, not redress; therefore, where officers acted in good-faith reliance on what they believed to be...
📊 First example labels sum: 1 traits detected


In [10]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')
print("✓ Tokenizer loaded")

# Initialize model (make sure NUM_TRAITS is defined)
NUM_TRAITS = 50

print("Initializing model...")
model = JudicialBiasClassifier(NUM_TRAITS)
model.to(device)
print("✓ Model initialized and moved to device")
print(f"✓ Model has {sum(p.numel() for p in model.parameters()):,} parameters")

class JudicialDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.FloatTensor(self.labels[idx])
        }

# Create dataset and dataloader
print("Creating dataset...")
dataset = JudicialDataset(texts, labels, tokenizer)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)
print(f"✓ Created DataLoader with {len(train_loader)} batches")

Using device: cuda
Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: nlpaueb/legal-bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.decoder.bias               | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.decoder.weight             | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✓ Model initialized and moved to device
✓ Model has 109,520,690 parameters
Creating dataset...
✓ Created DataLoader with 313 batches


In [11]:
import torch.nn as nn

# Training setup
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()
num_epochs = 3

print("🚀 Starting training...")
print("="*60)

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    batch_count = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        batch_count += 1

        # Print progress every 500 batches
        if batch_count % 500 == 0:
            print(f"  Epoch {epoch+1}/{num_epochs} | Batch {batch_count}/{len(train_loader)} | Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"\n{'='*60}")
    print(f"✅ Epoch {epoch+1}/{num_epochs} Complete | Avg Loss: {avg_loss:.4f}")
    print(f"{'='*60}\n")

# Save the trained model
torch.save(model.state_dict(), 'judicial_bias_model.pt')
print("💾 Model training complete!")
print("💾 Model saved as 'judicial_bias_model.pt'")

🚀 Starting training...

✅ Epoch 1/3 Complete | Avg Loss: 0.2006


✅ Epoch 2/3 Complete | Avg Loss: 0.1077


✅ Epoch 3/3 Complete | Avg Loss: 0.0955

💾 Model training complete!
💾 Model saved as 'judicial_bias_model.pt'


  '''
    # ─── STEP 1: Build Training Dataset ─────────────────────
    print("="*60)
    print(" STEP 1: BUILD DATASET")
    print("="*60)
    
    builder = JudicialDatasetBuilder()
    dataset = builder.build_complete_dataset()
    
    # Save dataset
    with open("judicial_training_data.json", "w") as f:
        json.dump(dataset, f, indent=2)

In [13]:

# ============================================================
# USAGE EXAMPLE
# ============================================================
if __name__ == "__main__":
  print(" STEP 2: TRAIN MODEL")
    # ─── STEP 3: Analyze Judges ─────────────────────────────
  print("\n" + "="*60)
  print(" STEP 3: ANALYZE JUDICIAL OPINIONS")
  print("="*60)

  profiler = JudgeProfiler(model_path="judicial_bias_model.pt")

  # Example: Analyze a single opinion
  opinion_text = """
  The defendant's criminal history, spanning over two decades,
  demonstrates a pattern of recidivist behavior that cannot be
  overlooked. While the defense argues for leniency based on
  the defendant's difficult upbringing, this Court finds that
  personal circumstances do not excuse repeated violations of law.
  The interests of public safety must prevail.
  """

  result = profiler.analyze_opinion(opinion_text)
  print("\nSingle Opinion Analysis:")
  print(f"  Detected traits: {list(result['traits'].keys())}")
  print(f"  Profile: {result['profile_summary']}")
  print(f"  Red flags: {result['red_flags']}")


  # Example: Analyze judge's entire corpus
  judge_opinions = [
      "Opinion 1 text...",
      "Opinion 2 text...",
      # ... load all opinions by this judge
  ]

  judge_profile = profiler.analyze_judge_corpus(judge_opinions)
  print("\n\nJudge Aggregate Profile:")
  print(f"  Consistent patterns: {judge_profile['consistent_patterns'].keys()}")


  # Example: Export to CSV
  all_results = {
      "case_001": profiler.analyze_opinion("text1"),
      "case_002": profiler.analyze_opinion("text2"),
  }

  export_analysis_to_csv(all_results, "judge_bias_analysis.csv")


  print("\n" + "="*60)
  print(" ✅ JUDICIAL BIAS CLASSIFIER READY")
  print("="*60)

 STEP 2: TRAIN MODEL

 STEP 3: ANALYZE JUDICIAL OPINIONS


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: nlpaueb/legal-bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.decoder.bias               | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.decoder.weight             | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Single Opinion Analysis:
  Detected traits: []
  Profile: No significant bias patterns detected.
  Red flags: []


Judge Aggregate Profile:
  Consistent patterns: dict_keys([])
✅ Exported to judge_bias_analysis.csv
   2 opinions analyzed
   50 trait columns

 ✅ JUDICIAL BIAS CLASSIFIER READY
